POSTagging dengan metode HMM-Viterbi. 

Contoh yang digunakan pada tutorial ini sesuai dengan contoh yang diberikan pada slide materi POSTagging. Asumsi yang digunakan adalah tabel emission dan transition probability sudah diketahui

Inisialisasi tagset

In [6]:
tags = {0:'<start>',1:'N',2:'V'}
print(tags)



{0: '<start>', 1: 'N', 2: 'V'}


Inisialisasi emission probability

In [9]:
emission_prob = {}

emission_prob[('N','deal')] = 0.2
emission_prob[('N','fail')] = 0.05
emission_prob[('N','talks')] = 0.2

emission_prob[('V','deal')] = 0.3
emission_prob[('V','fail')] = 0.3
emission_prob[('V','talks')] = 0.3

Inisialisasi transition probability

In [10]:
transition_prob = {}

transition_prob[('<start>','N')] = 0.8
transition_prob[('<start>','V')] = 0.2

transition_prob[('N','N')] = 0.4
transition_prob[('N','V')] = 0.6

transition_prob[('V','N')] = 0.8
transition_prob[('V','V')] = 0.2

Fungsi Viterbi

In [79]:
def viterbi(trans_prob, emission_prob, tokens):
    # create a path probability matrix viterbi[N,T]
    # N: banyaknya state
    # T: jumlah token

    T, N = len(tokens)+1, len(tags) # token ditambah satu untuk start
    new_tokens = ['<s>'] + tokens
    print('T=',T,',N=',N)
    print('token:',new_tokens)
    viterbi_mat = [[0 for x in range(T)] for y in range(N)] 
    print('matriks viterbi setelah di-create:',viterbi_mat)
    print('baris adalah tag/state dan kolom adalah token\n')
    # create backpointers matrix
    backpointers = [[0 for x in range(T)] for y in range(N)] 

    # initial probability distribution over states (phi)
    # transition probability dengan previous state adalah <start>
    phi = {}
    for i in range (1,len(tags)):
        phi[tags[i]] = transition_prob[('<start>',tags[i])]
                    
    # initialization
    # urutan index state sesuai dengan index di dictionary tags{}
    # inisialisasi dimulai dari state ke-1, state ke-0 sudah pasti <start>
    viterbi_mat[0][0] = 1.0 # untuk token <s>, tag = <start>
    for s in range(1,N):
        viterbi_mat[s][1] = phi[tags[s]] * emission_prob[(tags[s],new_tokens[1])]
        backpointers[s][1] = 0

    print('viterbi mat setelah inisialisasi, proses token pertama ',new_tokens[1],':')
    print(viterbi_mat)
    print('\n')

    # recursion step
    for t in range(2,T):
        print('token ke ',t,':',new_tokens[t])
        for s in range(1,N):
            # get max viterbi from previous transition
            max_prev_transition = 0.0
            max_state = 0
            print('menghitung nilai viterbi untuk state:',tags[s])
            for i in range(1,N):                
                #selain transisi dari tag <start>, range mulai dari indeks 1
                temp_transition = viterbi_mat[i][t-1] * transition_prob[(tags[i],tags[s])]
                print('t-1 = ',t-1)   
                print('current calculation ',tags[i],'i:',i,'=',viterbi_mat[i][t-1],'*',transition_prob[(tags[i],tags[s])],'=',temp_transition)           
                if temp_transition > max_prev_transition:
                    max_prev_transition = temp_transition
                    max_state = i
            viterbi_mat[s][t] = max_prev_transition * emission_prob[(tags[s],new_tokens[t])]
            backpointers[s][t] = max_state
        print('viterbi mat setelah proses token: ',new_tokens[t])
        print(viterbi_mat)
        print('\n')

    

    # terminasi
    # get max probability in last column
    max_last_prob = 0.0
    best_last_tag = ''
    idx_best_last_tag = 0
    for i in range (1,N):
        if viterbi_mat[i][T-1] > max_last_prob:
            max_last_prob = viterbi_mat[i][T-1]
            best_last_tag = tags[i]
            idx_best_last_tag = i
    print('last token = ',new_tokens[T-1],',tag = ',best_last_tag,',max_last_prob =',max_last_prob)

    best_path = []
    best_path.append(idx_best_last_tag)
    for i in range(T-1,1,-1):
        best_prev_tag = backpointers[idx_best_last_tag][i]
        print('best_prev_tag=',best_prev_tag)
        best_path.append(best_prev_tag)
    # reverse the order
    best_path = best_path[::-1]
    
    return viterbi_mat, best_path

Jalankan fungsi Viterbi untuk sebuah contoh kalimat 'deal talks fail'

In [80]:
viterbi_mat, best_path =  viterbi(transition_prob,emission_prob,['deal','talks','fail'])

T= 4 ,N= 3
token: ['<s>', 'deal', 'talks', 'fail']
matriks viterbi setelah di-create: [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
baris adalah tag/state dan kolom adalah token

viterbi mat setelah inisialisasi, proses token pertama  deal :
[[1.0, 0, 0, 0], [0, 0.16000000000000003, 0, 0], [0, 0.06, 0, 0]]


token ke  2 : talks
menghitung nilai viterbi untuk state: N
t-1 =  1
current calculation  N i: 1 = 0.16000000000000003 * 0.4 = 0.06400000000000002
t-1 =  1
current calculation  V i: 2 = 0.06 * 0.8 = 0.048
menghitung nilai viterbi untuk state: V
t-1 =  1
current calculation  N i: 1 = 0.16000000000000003 * 0.6 = 0.09600000000000002
t-1 =  1
current calculation  V i: 2 = 0.06 * 0.2 = 0.012
viterbi mat setelah proses token:  talks
[[1.0, 0, 0, 0], [0, 0.16000000000000003, 0.012800000000000004, 0], [0, 0.06, 0.028800000000000003, 0]]


token ke  3 : fail
menghitung nilai viterbi untuk state: N
t-1 =  2
current calculation  N i: 1 = 0.012800000000000004 * 0.4 = 0.005120000000000002
t-1 =  2

Tes print viterbi matrix yang terbentuk

In [82]:
# tes print viterbi matrix
print('test print viterbi matrix final')
rows, columns = len(viterbi_mat), len(viterbi_mat[0])
for row_index in range(0,rows):
    print('row index =',row_index)
    print(viterbi_mat[row_index])


test print viterbi matrix final
row index = 0
[1.0, 0, 0, 0]
row index = 1
[0, 0.16000000000000003, 0.012800000000000004, 0.0011520000000000002]
row index = 2
[0, 0.06, 0.028800000000000003, 0.0023040000000000005]


Tes print best POSTag yang diperoleh

In [83]:
str_tag = ''
for tag in best_path:
    str_tag += tags[tag]+' '
print('POSTag untuk kalimat deal talks fail:',str_tag)

POSTag untuk kalimat deal talks fail: N N V 
